In [ ]:
%load_ext autoreload
%autoreload 2

# 確保可以從專案根目錄匯入模組
current_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(current_dir, "../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
import sys
from typing import List
import requests
from requests.exceptions import ConnectionError, Timeout

# 檢查和 Google 翻譯伺服器的連線
try:
    response = requests.get('https://translate.google.com', timeout=10)
    response.raise_for_status() # 如果狀態碼不是 200，會引發 HTTPError
    print("網路連線正常，可以存取 Google 翻譯服務。")
except Timeout:
    print("連線超時，可能是網路速度太慢或防火牆阻擋。")
except ConnectionError:
    print("連線失敗，請檢查網路連線或代理伺服器設定。")
except Exception as e:
    print(f"發生其他連線錯誤: {e}")

# 檢查 deep_translator 模組是否成功安裝
try:
    from deep_translator import GoogleTranslator as _GT
except Exception:
    _GT = None
    print("deep_translator 模組未安裝，部分功能可能無法使用。")
print(f"GoogleTranslator 物件: {_GT}")



In [ ]:
# 測試主要function   
from etl_showcase.infrastructure.cleaning.language_translator import LanguageTranslator

# 初始化翻譯器，指定目標變體為繁體中文
translator = LanguageTranslator()

# 準備測試資料
texts_to_test = [
"あなたの本性、この1問でバレます",
"top 6 Love Psychology Facts | Signs Someone Truly Loves You | Relationship Tips & Attraction Secret!",
"别人的速度，不是你的节奏 #理性清醒#自我成长#别再比较了#人生不是比赛#做更好的自己#心理建设#成长笔记",
"從MIT到出家路，證明我不是失敗者#自我成長#人生感悟#出家 #經歷",
]

# 偵測語言測試
print("--- 語言偵測測試 ---")
detected_langs = translator.batch_detect_lang(texts_to_test)
for text, lang in zip(texts_to_test, detected_langs):
    print(f"'{text}' => 偵測語言: {lang}")

# 批量翻譯測試
print("\n--- 批量翻譯測試 ---")

# 這裡只進行翻譯，不包含繁簡轉換
translated_texts = translator.batch_translate(texts_to_test, target_language="zh-TW")
for text, translated in zip(texts_to_test, translated_texts):
    print(f"'{text}' => 翻譯結果: {translated}")

# 額外測試：簡體轉繁體
# 此步驟模擬 youtube_data_preprocessor.py 中的邏輯
print("\n--- 中文變體轉換測試 (簡轉繁) ---")
zh_texts = [
"测试一下简体中文转换",
"软件开发是一门艺术",
"人工智能的发展日新月异"
]
translated_zh_texts = [translator.to_target_chinese_variant(text, target_variant="zh-TW") for text in zh_texts]
for text, converted in zip(zh_texts, translated_zh_texts):
    print(f"'{text}' => 轉換結果: {converted}")